# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sentyabrskiy,-16.5048,-68.1624,6.18,93,75,3.09,BO,"Feb 07, 2023, Tue, 23:00"
1,1,el alto,-23.1230,-134.9680,26.34,73,67,3.78,PF,"Feb 07, 2023, Tue, 23:00"
2,2,rikitea,-20.7724,117.1458,34.47,43,54,6.84,AU,"Feb 07, 2023, Tue, 23:00"
3,3,roebourne,1.7505,40.0595,24.97,73,20,4.12,KE,"Feb 07, 2023, Tue, 23:00"
4,4,wajir,-14.4646,132.2636,36.17,44,40,2.06,AU,"Feb 07, 2023, Tue, 23:00"


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    framde_width = 500,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)
# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] <= 27) & (city_data_df["Max Temp"] >= 21) & (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()
# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,ayorou,18.2353,-72.5375,24.01,72,0,2.07,HT,"Feb 07, 2023, Tue, 23:00"
259,259,san luis de la paz,22.9333,97.7500,25.44,22,0,1.45,MM,"Feb 07, 2023, Tue, 23:00"
314,314,bathsheba,-34.9087,-54.9583,21.91,83,0,3.60,UY,"Feb 07, 2023, Tue, 23:00"
326,326,cayenne,17.0175,54.1011,24.04,64,0,2.57,OM,"Feb 07, 2023, Tue, 23:00"
344,344,puerto maldonado,20.5592,-76.9165,22.12,84,0,4.37,CU,"Feb 07, 2023, Tue, 23:00"
367,367,palmer,-13.7100,-76.2030,22.94,83,0,3.09,PE,"Feb 07, 2023, Tue, 23:00"
438,438,elhovo,-31.4184,-64.4937,24.74,10,0,0.89,AR,"Feb 07, 2023, Tue, 23:00"
579,579,santa eulalia del rio,25.3833,97.4000,26.99,25,0,1.82,MM,"Feb 07, 2023, Tue, 23:00"


In [17]:
len(ideal_weather)

8

### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
                    
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df = hotel_df.reset_index()
hotel_df.head()

,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,0,sentyabrskiy,BO,-16.5048,-68.1624,93,
1,1,el alto,PF,-23.1230,-134.9680,73,
2,2,rikitea,AU,-20.7724,117.1458,43,
3,3,roebourne,KE,1.7505,40.0595,73,
4,4,wajir,AU,-14.4646,132.2636,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
radius = 10000
categories = "accommodation.hotel"

params = {
   "categories":categories,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sentyabrskiy - nearest hotel: Alojamiento
el alto - nearest hotel: Chez Bianca & Benoit
rikitea - nearest hotel: No hotel found
roebourne - nearest hotel: jeddah hospital
wajir - nearest hotel: No hotel found
katherine - nearest hotel: No hotel found
belaya gora - nearest hotel: Хромит
khromtau - nearest hotel: No hotel found
basco - nearest hotel: Cape Fox Lodge
ketchikan - nearest hotel: No hotel found
barrow - nearest hotel: Matai Lodge
te anau - nearest hotel: No hotel found
tukrah - nearest hotel: Sandman Signature Kamloops Hotel
kamloops - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
dikson - nearest hotel: 15 On Orange Hotel
cape town - nearest hotel: No hotel found
carnarvon - nearest hotel: Тасеевское
baley - nearest hotel: Hotel K.T. Mutiara
kuala terengganu - nearest hotel: Hampton
clyde river - nearest hotel: No hotel found
hunza - nearest hotel: Aloe guest house
hermanus - nearest hotel: Victoria Hotel
bredasdorp - nea

,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,0,sentyabrskiy,BO,-16.5048,-68.1624,93,Alojamiento
1,1,el alto,PF,-23.1230,-134.9680,73,Chez Bianca & Benoit
2,2,rikitea,AU,-20.7724,117.1458,43,No hotel found
3,3,roebourne,KE,1.7505,40.0595,73,jeddah hospital
4,4,wajir,AU,-14.4646,132.2636,44,No hotel found
...,...,...,...,...,...,...,...
627,627,bardiyah,AU,-27.1823,151.2634,32,No hotel found
628,628,kazalinsk,AU,-27.1823,151.2634,32,No hotel found
629,629,okha,AU,-27.1823,151.2634,32,No hotel found
630,630,kunming,AU,-27.1823,151.2634,32,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)